In [1]:
using Random
using PyCall
using Base.Iterators
using ScikitLearn

import ScikitLearn.fit!

## Load dataset

### Full Julia version
Still a work in progress needs some fixing

In [2]:
# function needs a bit of fixing will come back to it later, currently using tht pycall version to load dataset

function load_planar_dataset()
    Random.seed!(1)
    m = 400 # number of examples
    N = Int(m/2) # number of points per class
    D = 2 # dimensionality
    X = zeros((m, D))
    Y = zeros(UInt8, (m, 1))
    a = 4
    
    for j in UnitRange(1, 2)
        ix = UnitRange(N*j+1, N*(j+1))
        @show size(ix), size(X), size(Y), ix
        t = LinRange(j*3.12, (j+1)*3.12, N) .+ randn(N) .* 0.2
        r = a .* sin.(4 .* t) .+ randn(N) .* 0.2
        X[ix] = hcat.(r .* sin.(t), r .* cos.(t))
        Y[ix] = j
    end
        
    X = X'
    Y = Y'
        
    return X, Y
end

load_planar_dataset (generic function with 1 method)

### PyCall version
A little bit of cheating 😜😝. But I promise I will fix it.

In [3]:
function load_planar_dataset_pycall()
    py"""
    import numpy as np
    
    def load_planar_dataset():
        np.random.seed(1)
        m = 400 # number of examples
        N = int(m/2) # number of points per class
        D = 2 # dimensionality
        X = np.zeros((m,D)) # data matrix where each row is a single example
        Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
        a = 4 # maximum ray of the flower

        for j in range(2):
            ix = range(N*j,N*(j+1))
            t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
            r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius    
            X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
            Y[ix] = j

        X = X.T
        Y = Y.T

        return X, Y
    """
    X, Y = py"load_planar_dataset()"
    
    return X, Int.(Y)
end

load_planar_dataset_pycall (generic function with 1 method)

In [4]:
py"""
def load_extra_datasets():  
    N = 200
    noisy_circles = sklearn.datasets.make_circles(n_samples=N, factor=.5, noise=.3)
    noisy_moons = sklearn.datasets.make_moons(n_samples=N, noise=.2)
    blobs = sklearn.datasets.make_blobs(n_samples=N, random_state=5, n_features=2, centers=6)
    gaussian_quantiles = sklearn.datasets.make_gaussian_quantiles(mean=None, cov=0.5, n_samples=N, n_features=2, n_classes=2, shuffle=True, random_state=None)
    no_structure = np.random.rand(N, 2), np.random.rand(N, 2)
    
    return noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure
"""

In [5]:
X, y = load_planar_dataset_pycall()

([1.204442292254319 0.15870990398255372 … 0.3243868928570292 0.7307808250007513; 3.5761141481288448 -1.4821709003683443 … -2.116324663250665 3.0683292081876545], [0 0 … 1 1])

In [6]:
@sk_import linear_model: LogisticRegression

PyObject <class 'sklearn.linear_model._logistic.LogisticRegression'>

In [7]:
model = LogisticRegression(fit_intercept=true)

PyObject LogisticRegression()

In [8]:
fit!(model, X', y')

C:\Users\Abhishek Swain\miniconda3\envs\abhishek\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


PyObject LogisticRegression()

In [6]:
function σ(z) 
    """
    Compute the sigmoid of z
    """
    return one(z) / (one(z) + exp(-z))
end

σ (generic function with 1 method)

## Neural Network model

Logistic regression did not work well on the "flower dataset". You are going to train a Neural Network with a single hidden layer.

**Here is our model**:
<img src="classification_kiank.png" style="width:600px;height:300px;">

**Mathematically**:

For one example $x^{(i)}$:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{1}$$ 
$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

Given the predictions on all the examples, you can also compute the cost $J$ as follows: 
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

**Reminder**: The general methodology to build a Neural Network is to:
    1. Define the neural network structure ( # of input units,  # of hidden units, etc). 
    2. Initialize the model's parameters
    3. Loop:
        - Implement forward propagation
        - Compute loss
        - Implement backward propagation to get the gradients
        - Update parameters (gradient descent)

You often build helper functions to compute steps 1-3 and then merge them into one function we call `nn_model()`. Once you've built `nn_model()` and learnt the right parameters, you can make predictions on new data.

### Defining the neural network structure ####

**Exercise**: Define three variables:
    - n_x: the size of the input layer
    - n_h: the size of the hidden layer (set this to 4) 
    - n_y: the size of the output layer

**Hint**: Use shapes of X and Y to find n_x and n_y. Also, hard code the hidden layer size to be 4.

In [7]:
function layer_sizes(X, y)
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    
    n_x = size(X, 1)
    n_h = 4
    n_y = size(y, 1)
end

layer_sizes (generic function with 1 method)

### Initialize the model's parameters ####

**Exercise**: Implement the function `initialize_parameters()`.

**Instructions**:
- Make sure your parameters' sizes are right. Refer to the neural network figure above if needed.
- You will initialize the weights matrices with random values. 
    - Use: `Random.randn(a,b) .* 0.01` to randomly initialize a matrix of shape (a,b).
- You will initialize the bias vectors as zeros. 
    - Use: `zeros((a,b))` to initialize a matrix of shape (a,b) with zeros.

In [8]:
function initialize_parameters(n_x, n_h, n_y)
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- Julia dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    Random.seed!(2)
    
    W¹ = Random.randn(n_h, n_x) .* 0.01
    b¹ = zeros(n_h, 1)
    W² = Random.randn(n_y, n_h) .* 0.01
    b² = zeros(n_y, 1)
    
    @assert(size(W¹) == (n_h, nx))
    @assert(size(b¹) == (n_h, 1))
    @assert(size(W²) == (n_y, n_h))
    @assert(size(b²) == (n_y, 1))
    
    parameters = Dict(
        "W¹" => W¹,
        "b¹" => b¹,
        "W²" => W²,
        "b²" => b²
    )
    
    return parameters;
end

initialize_parameters (generic function with 1 method)

### The Loop ####

**Question**: Implement `forward_propagation()`.

**Instructions**:
- Look above at the mathematical representation of your classifier.
- You can use the function `sigmoid()`. It is built-in (imported) in the notebook.
- You can use the function `np.tanh()`. It is part of the numpy library.
- The steps you have to implement are:
    1. Retrieve each parameter from the dictionary "parameters" (which is the output of `initialize_parameters()`) by using `parameters[".."]`.
    2. Implement Forward Propagation. Compute $Z^{[1]}, A^{[1]}, Z^{[2]}$ and $A^{[2]}$ (the vector of all your predictions on all the examples in the training set).
- Values needed in the backpropagation are stored in "`cache`". The `cache` will be given as an input to the backpropagation function.

In [9]:
function forward_propagation(X, parameters)
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- Julia dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    
    # Retieve parameters
    W¹ = parameters["W¹"]
    b¹ = parameters["b¹"]
    W² = parameters["W²"]
    b² = parameters["b²"]
    
    # Forward propagation 
    Z¹ = W¹'X + b¹
    A¹ = σ.(Z¹)
    Z² = W¹'A¹ + b²
    A² = σ.(Z²)
    
    @assert(size(A²) == (1, size(X, 2)))
    
    cache = Dict(
        "Z¹" => Z¹,
        "A²" => A²,
        "Z²" => Z²,
        "A²" => A²
    )
    
    return cache;
end

forward_propagation (generic function with 1 method)

Now that you have computed $A^{[2]}$ (in the Python variable "`A2`"), which contains $a^{[2](i)}$ for every example, you can compute the cost function as follows:

$$J = - \frac{1}{m} \sum\limits_{i = 1}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small\tag{13}$$

**Exercise**: Implement `compute_cost()` to compute the value of the cost $J$.

**Instructions**:
- There are many ways to implement the cross-entropy loss. To help you, we give you how we would have implemented
$- \sum\limits_{i=0}^{m}  y^{(i)}\log(a^{[2](i)})$:
```python
logprobs = np.multiply(np.log(A2),Y)
cost = - np.sum(logprobs)                # no need to use a for loop!
```

(you can use either `np.multiply()` and then `np.sum()` or directly `np.dot()`).  
Note that if you use `np.multiply` followed by `np.sum` the end result will be a type `float`, whereas if you use `np.dot`, the result will be a 2D numpy array.  We can use `np.squeeze()` to remove redundant dimensions (in the case of single float, this will be reduced to a zero-dimension array). We can cast the array as a type `float` using `float()`.

In [10]:
function compute_cost(A², y, parameters)
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    [Note that the parameters argument is not used in this function, 
    but the auto-grader currently expects this parameter.
    Future version of this notebook will fix both the notebook 
    and the auto-grader so that `parameters` is not needed.
    For now, please include `parameters` in the function signature,
    and also when invoking this function.]
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    
    """
    
    m = size(y, 2)
    
    # Compute cost
    𝒥 = -1 * sum(y .* log.(A²) .+ (1 .- y) .* log.(1 .- A²))
    𝒥 /= m
    
    @assert(isa(𝒥, Float64))
    
    return 𝒥
end

compute_cost (generic function with 1 method)

Using the cache computed during forward propagation, you can now implement backward propagation.

**Question**: Implement the function `backward_propagation()`.

**Instructions**:
Backpropagation is usually the hardest (most mathematical) part in deep learning. To help you, here again is the slide from the lecture on backpropagation. You'll want to use the six equations on the right of this slide, since you are building a vectorized implementation.  

<img src="grad_summary.png" style="width:600px;height:300px;">

<!--
$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (a^{[2](i)} - y^{(i)})$

$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $

$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$

$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * ( 1 - a^{[1] (i) 2}) $

$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $

$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$

- Note that $*$ denotes elementwise multiplication.
- The notation you will use is common in deep learning coding:
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$
    
!-->

- Tips:
    - To compute dZ1 you'll need to compute $g^{[1]'}(Z^{[1]})$. Since $g^{[1]}(.)$ is the tanh activation function, if $a = g^{[1]}(z)$ then $g^{[1]'}(z) = 1-a^2$. So you can compute 
    $g^{[1]'}(Z^{[1]})$ using `(1 - np.power(A1, 2))`.

In [11]:
function backward_propagation(parameters, cache, X, y)
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    
    m = size(X, 2)
    
    W¹ = parameters["W¹"]
    W² = parameters["W²"]
    
    A¹ = cache["A¹"]
    A² = cache["A²"]
    
    𝜕𝑍² = A² - y
    𝜕𝑊² = (1/m) .* 𝜕𝑍²A¹'
    𝜕𝑏² = (1/m) * sum(𝜕𝑍², 1)
    𝜕𝑍¹ = W²'𝜕𝑍² .* (1 .- A¹ .^ 2)
    𝜕𝑊¹ = (1/m) .* 𝜕Z¹ * X'
    𝜕𝑏¹ = (1/m) * sum(𝜕𝑍¹, 1)
    
    grads = Dict(
        "𝜕𝑏²" => 𝜕𝑏²,
        "𝜕𝑊²" => 𝜕𝑊²,
        "𝜕𝑏¹" => 𝜕𝑏¹,
        "𝜕𝑊¹" => 𝜕𝑊¹
    )
    
    return grads
    
end

backward_propagation (generic function with 1 method)

In [12]:
function update_parameters(parameters, grads, 𝛼 = 1.2)
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    
    W¹ = parameters["W¹"]
    b¹ = parameters["b¹"]
    W² = parameters["W²"]
    b² = parameters["b²"]
    
    𝜕𝑊¹ = grads["𝜕𝑊¹"]
    𝜕𝑏¹ = grads["𝜕𝑏¹"]
    𝜕𝑊² = grads["𝜕𝑊²"]
    𝜕𝑏² = grads["𝜕𝑏²"]
    
    W¹ = W¹ - 𝛼 .* 𝜕𝑊¹
    b¹ = b¹ - 𝛼 .* 𝜕𝑏¹
    W² = W² - 𝛼 .* 𝜕𝑊²
    b² = b² - 𝛼 .* 𝜕𝑏²

    parameters = Dict(
        "W¹" => W¹,
        "b¹" => b¹,
        "W²" => W²,
        "b²" => b²
    )
    
    return parameters
end

update_parameters (generic function with 2 methods)

### Integrate all parts in nn_model() ####

**Question**: Build your neural network model in `nn_model()`.

**Instructions**: The neural network model has to use the previous functions in the right order.

In [13]:
function nn_model(X, Y, n_h, num_iterations = 10000, print_cost=false)
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    Random.seed!(3)
    n_x = layer_sizes(X, Y)[1]
    n_y = layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    for i=1:num_iterations
        A², cache = forward_propagation(X, parameters)
        
        cost = compute_cost(A², Y, parameters)
        
        grads = backward_propagation(parameters, cache, X, Y)
        
        parameters = update_parameters(parameters, grads)
        
        if print_cost && i % 1000 == 0
            println("Cost after iteration $i: %cost")
        end
    end
    
    return parameters
end

nn_model (generic function with 3 methods)

In [14]:
function nn_predict(parameters, X)
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    m = size(X, 2)
    predictions = zeros(1, m)
    
    A², cache = forward_propagation(X, parameters)
    predictions = [p > 0.5 ? 1 : 0 for p in Iterators.flatten(A²)]
    
    return predictions
end

nn_predict (generic function with 1 method)

In [16]:
parameters = nn_model(X', y', 4)

BoundsError: BoundsError